In [2]:
import sys
sys.path.append("/home/jarlehti/projects/gradu")

In [3]:
import os
import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
import d3p
from src.napsu_mq.napsu_mq import NapsuMQModel, NapsuMQResult

In [4]:
CURRENT_FOLDER = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
DATASETS_FOLDER = os.path.join(CURRENT_FOLDER, "data", "datasets")
MODELS = os.path.join(CURRENT_FOLDER, "models")

In [5]:
rng = jax.random.PRNGKey(23453252353)

In [6]:
dataset = pd.read_csv(os.path.join(DATASETS_FOLDER, "binary4d.csv"), dtype="category")
n, d = dataset.shape
print(n)
print(d)

100000
4


In [7]:
column_feature_set = [
    ('A', 'D'), 
    ('B', 'D'), 
    ('C', 'D'), 
]

model = NapsuMQModel()
result = model.fit(
    data=dataset,
    dataset_name="binary4d",
    rng=rng,
    epsilon=8,
    delta=(n ** (-2)),
    column_feature_set=column_feature_set,
    use_laplace_approximation=True,
    laplace_approximation_algorithm="torch_LBFGS",
)

No experiment_id found: <ContextVar name='experiment_id' at 0x7f097d017860>
Setting experiment_id to MDJ7H8OL
Domain size: 16
Recording:  Query selection
start MST selection


/home/jarlehti/projects/gradu/src/napsu_mq/mst.py:130: RuntimeWarning: divide by zero encountered in double_scalars
  epsilon = np.sqrt(8 * rho / (r - 1))


end MST selection
Recording:  Calculating full marginal query
MST query set: [('A', 'D'), ('B', 'D'), ('C', 'D')]
Recording:  Calculating canonical query set
Calculating canonical queries, clique_set length: 8


  0%|                                                                                                     | 0/8 [00:00<?, ?it/s]
4it [00:00, 3228.25it/s]

2it [00:00, 1913.46it/s]

4it [00:00, 5174.96it/s]

4it [00:00, 5329.48it/s]

2it [00:00, 3142.98it/s]

2it [00:00, 3881.82it/s]

2it [00:00, 2986.33it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 14154.01it/s]


Calculating new queries, not_original_clique_queries length: 4


100%|██████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 10217.55it/s]


Canonical queries: 7
Junction tree width: 2
(7,)
2.0428480633449504
DP noise mean: -0.6310864307234916
[38048.98081449 11568.60055397 42244.80231782  7815.86096104
 34386.26226323 45831.54523827  4156.53024617]
SUFF STAT MEANS
26293.85714285714
26293.226056426418
SUFF STAT VARIANCES
269748590.6938776
269736375.01463854
Recording:  Laplace approximation
Running Laplace optimization
Laplace approximation time: {'experiment_id': 'MDJ7H8OL', 'start': 1774926.545500524, 'stop': 1774927.246458368, 'timedelta': 0.7009578438010067, 'task': 'Laplace approximation', 'dataset_name': 'binary4d', 'query_str': 'A+D-B+D-C+D', 'query_list': [('A', 'D'), ('B', 'D'), ('C', 'D')], 'epsilon': 8, 'delta': 1e-10, 'MCMC_algo': 'NUTS', 'laplace_approximation': True, 'missing_query': None, 'discretization': None, 'no_privacy': False, 'n_canonical_queries': 7, 'junction_tree_width': 2, 'suff_stat_dim': (7,), 'suff_stat': array([38047, 11570, 42246,  7815, 34388, 45834,  4157]), 'dp_suff_stat': DeviceArray([3804

/home/jarlehti/projects/gradu/src/napsu_mq/maximum_entropy_inference.py:168: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = numpyro.infer.MCMC(


KeyboardInterrupt: 

In [7]:
result_path = os.path.join(MODELS, "napsu_testing_binary4d_all_queries_500e_NUTS_LA.dill")
result.store(result_path)

In [8]:
napsu_result_read_file = open(os.path.join(MODELS, "napsu_testing_binary4d_all_queries_500e_NUTS_LA.dill"), "rb")
loaded_result = NapsuMQResult.load(napsu_result_read_file)
sampling_rng = jax.random.PRNGKey(234513465234)
datasets = loaded_result.generate_extended(rng=sampling_rng, num_data_per_parameter_sample=2000, num_parameter_samples=5)

Generating data with 2000 points and 5 datasets


In [9]:
datasets

[      A  B  C  D
 0     0  0  1  1
 1     1  0  1  1
 2     0  1  0  1
 3     1  1  1  1
 4     1  0  1  1
 ...  .. .. .. ..
 1995  0  0  1  1
 1996  1  1  1  1
 1997  0  0  0  0
 1998  1  1  1  1
 1999  0  1  0  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0     0  1  1  1
 1     1  0  1  1
 2     0  0  0  0
 3     0  0  1  1
 4     1  0  0  0
 ...  .. .. .. ..
 1995  1  0  0  1
 1996  0  1  1  1
 1997  1  0  0  1
 1998  0  1  0  0
 1999  1  1  1  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0     1  1  1  1
 1     0  0  1  1
 2     1  0  1  1
 3     0  1  1  1
 4     0  1  1  1
 ...  .. .. .. ..
 1995  0  0  1  1
 1996  1  1  1  1
 1997  1  1  1  1
 1998  1  0  0  1
 1999  0  1  1  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0     0  1  1  0
 1     0  1  1  1
 2     0  1  0  1
 3     0  1  0  1
 4     1  0  1  1
 ...  .. .. .. ..
 1995  1  0  1  1
 1996  0  0  0  0
 1997  1  1  1  1
 1998  1  0  0  1
 1999  0  0  1  1
 
 [2000 rows x 4 columns],
       A  B  C  D
 0    